## ELIZA

Copyright (C) 2019 Szymon Jessa

### Kod Elizy

Importujemy biblioteki:

In [0]:
import doctest
import re

Tworzymy zmienną globalną, która będzie zapisywała wypowiedzi podczas konwersacji. 

In [0]:
memstack = []

Funkcja odpowiadająca za przetworzenie wypowiedzi użytkownika i zaproponowanie odpowiedzi.

In [0]:
def get_keystack(user_input):
    # scan all words in the utterance for keywords one-by-one
    # and put tuples (keyword, rank) in the list
    # ***CHALLENGE*** modify the code to handle UPPERCASE LETTERS and punctuations
    keystack = []
    for w in user_input.split():
        if w in script:
            keystack.append((w, script[w].get("rank", 0)))

    # sort keywords by rank, descending
    keystack = sorted(keystack, key=lambda i: i[1])

    # extract keywords from tuples
    keystack = [w for w, r in keystack]

    return keystack

Funkcja "zapamiętująca" słowa kluczowe pojawiające się we wcześniejszej rozmowie:

In [0]:
def memorize_user_input(user_input, user_input_trans):
    # scan all words for memory keywords
    memory_keywords = []
    for w in user_input.split():
        if w in script_memory:
            memory_keywords.append(w)
    memory_keywords = list(set(memory_keywords))

    for k in memory_keywords:
        # use first reassembly rule
        # ***CHALLENGE*** modify the code to use more than one reassembly rule!
        memresp = re.sub(script_memory[k]["decomposition"], script_memory[k]["reassembly"][0], user_input_trans)
        memstack.append(memresp)



Funkcja wybierająca odpowiedź Elizy:

In [0]:
def process(user_input):
    """ (str) -> str
    Return Eliza's answer for given input
    Return responses associated with the matched keyword
    Return responses for "none" keyword or use memory if other keywords not found
    """

    # extract keywords
    keystack = get_keystack(user_input)

    # transform user input using substitutions
    user_input_trans = " ".join(map(lambda w: substitutions.get(w, w), user_input.split()))

    # find a response based on the keywords
    resp = ""
    if keystack:
        # get response associated with the highest ranked keyword
        kw = keystack[-1] # get top ranked keyword
        rule = script[kw] # get script rule (decomposition & reassembly) for this keyword
        # check if the decomposition rule cane be applied for the utterance 
        if re.search(rule["decomposition"], user_input_trans):
            # if yes, generate response using the first reassembly rule
            trans = rule["reassembly"].pop(0) # pop first reassembly from list
            rule["reassembly"].append(trans) # append the reassembly rule to the end of the list
            resp = re.sub(rule["decomposition"], trans, user_input_trans) # generate response

    # if none keywords or rules were found, try to use memory
    if resp == "":
        if memstack:
            resp = memstack.pop(0)
        # if there are no responses in memory,
        # use default responses associated with special keyword "none"
        else:
            # ***CHALLENGE*** modify the code to support more than one default answer (reassembly rule)
            # ***CHALLENGE*** ensure the code won't break if "none" key doesn't exist
            resp = script["none"]["reassembly"][0]

    # if possible, generate new responses to be stored in memory for later
    memorize_user_input(user_input, user_input_trans)

    return str(resp)

Funkcja do uruchomiania czatu:

In [0]:
def chat():
    """ () -> None
    Allows interaction with Eliza in a loop:
    - read input from console
    - get Eliza's response using process function
    - write Eliza's response to console
    - exit if input string length is 0
    """
    
    print("<hit enter with no input to exit>")
    print("Eliza: How do you do. Please tell me your problem")
    while True:
        utt = input("Student: ")
        if not utt:
            break

        resp = process(utt)
        print("Eliza: %s" % resp)
        print(memstack)

### Testy

Tutaj puszczamy testy dla konkretnych zapytań dla Elizy. Doc testy puszczają dany kod oznaczony ```>>>``` i oczekują odpowiedzi zgodnej z tym, co jest poniżej. Oto przykład:

In [0]:
def task0_true():
  """
  >>> 2+2
  4
  """

In [0]:
def task0_false():
  """
  >>> 2+2
  5
  """

In [9]:
doctest.run_docstring_examples(task0_true, globals(), name="task0", verbose=True)

Finding tests in task0
Trying:
    2+2
Expecting:
    4
ok


In [10]:
doctest.run_docstring_examples(task0_false, globals(), name="task0", verbose=True)

Finding tests in task0
Trying:
    2+2
Expecting:
    5
**********************************************************************
File "__main__", line 3, in task0
Failed example:
    2+2
Expected:
    5
Got:
    4


In [0]:
def task1():
    """
    >>> process("I have no problems")
    "Are you saying 'no' just to be negative?"
    >>> process("no")
    'You are being a bit negative'
    >>> process("no")
    'Why not'
    """
    pass

In [0]:
def task2():
    """
    >>> process("hmm")
    'I am not sure I understand you fully'
    """
    pass

In [0]:
def task3():
    """
    >>> process("no I am not a negative person")
    'Is it because you are not a negative person that you came to me?'
    >>> process("no")
    'You are being a bit negative'
    """
    pass

In [0]:
def task4():
    """
    >>> process("you are repeating yourself")
    'What makes you think I am repeating myself?'
    >>> process("you are kidding me")
    'Does it please you to believe I am kidding you?'
    """
    pass

In [0]:
def task5():
    """
    >>> process("my wife said I am optimistic")
    'Tell me more about your family'
    >>> process("now I am sad")
    'Is it because you are sad that you came to me?'
    >>> process("maybe")
    'But your wife said you are optimistic?'
    """
    pass

### Skrypt rozmowy

Tutaj znajduje się skrypt rozmowy. W słowniku 'decomposition' to szukane wyrażenia regularne, natomiast 'reassembly' to odpowiedź Elizy. 

In [0]:
script = {
    "I": {"decomposition": r".*(I am)(.*)",
           "reassembly": [r'Is it because you are\2 that you came to me?']},
    "no": {"decomposition": r".*",
           "reassembly": ["Are you saying 'no' just to be negative?", 'You are being a bit negative', 'Why not']},
    "yes": {"decomposition": r".*",
            "reassembly": ["Tell me more about it"]   },
    "none": {"decomposition": r".*",
            "reassembly": ['I am not sure I understand you fully']},
    "you": {"decomposition": r".*are(.*)",
            "reassembly": [r"What makes you think I am\1?", r"Does it please you to believe I am\1?"]},
    "wife": {"decomposition": r".*",
             "reassembly": ["Tell me more about your family"]}
          }

Zastępstwa - tutaj możemy zmieniać pewne wyrażenia, żeby płynniej prowadzić rozmowę.

In [0]:
substitutions = {"yourself":"myself","me":"you"}

Script memory - tutaj tworzymy skrypt rozmowy (j.w.) ale wykorzystując zapamiętane wcześniej słowa.

In [0]:
script_memory = {}

W przypadku niektórych testów poniżej mamy do czynienia z losowością, więc czasem trzeba powtórzyć wykonanie go, aby wynik był właściwy.

In [32]:
doctest.run_docstring_examples(task1, globals(), name="task1", verbose=True)

Finding tests in task1
Trying:
    process("I have no problems")
Expecting:
    "Are you saying 'no' just to be negative?"
ok
Trying:
    process("no")
Expecting:
    'You are being a bit negative'
ok
Trying:
    process("no")
Expecting:
    'Why not'
ok


In [37]:
doctest.run_docstring_examples(task2, globals(), name="task2", verbose=True)


Finding tests in task2
Trying:
    process("hmm")
Expecting:
    'I am not sure I understand you fully'
ok


In [56]:
doctest.run_docstring_examples(task3, globals(), name="task3", verbose=True)


Finding tests in task3
Trying:
    process("no, I am not a negative person")
Expecting:
    'Is it because you are not a negative person that you came to me?'
ok
Trying:
    process("no")
Expecting:
    'You are being a bit negative'
ok


In [75]:
doctest.run_docstring_examples(task4, globals(), name="task4", verbose=True)


Finding tests in task4
Trying:
    process("you are repeating yourself")
Expecting:
    'What makes you think I am repeating myself?'
ok
Trying:
    process("you are kidding me")
Expecting:
    'Does it please you to believe I am kidding you?'
ok


In [0]:
doctest.run_docstring_examples(task4, globals(), name="task5", verbose=True)

Tutaj można odpalić rozmowę z Elizą.

In [83]:
chat()

<hit enter with no input to exit>
Eliza: How do you do. Please tell me your problem
Student: no
Eliza: You are being a bit negative
[]
Student: yes
Eliza: Tell me more about it
[]


KeyboardInterrupt: ignored

In [73]:
memstack

[]